In [1]:
import torch
import torchvision.models as models
from torch import nn

vgg16_pretrained = models.vgg16(pretrained = False)

def decoder(input_channel, output_channel, num = 3):
    if num == 3:
        decoder_body = nn.Sequential(
            nn.ConvTranspose2d(input_channel, input_channel, 3, padding = 1),
            nn.ConvTranspose2d(input_channel, input_channel, 3, padding = 1),
            nn.ConvTranspose2d(input_channel, output_channel, 3, padding = 1)
        )
    
    elif num == 2:
        decoder_body = nn.Sequential(
            nn.ConvTranspose2d(input_channel, input_channel, 3, padding = 1),
            nn.ConvTranspose2d(input_channel, output_channel, 3, padding = 1)
        )

    
    return decoder_body

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


# Unpooling 과 DeConvolution의 차이

- Unpooling : 이미지 공간으로의 강한 Activation의 위치를 추적함으로써, 관련된 예시들을 더 잘아내는 역할을 수행함.

- Deconvolution : 원본 이미지 내의 noisy activatian을 잘 걸러내며, target과 관련된 부분의 활성화를 증폭함.

In [2]:
class VGG16_deconv(torch.nn.Module):
    def __init__(self, num_classes = 8, num_linear = 131072, channel = 512, height = 16, width = 16):
        super(VGG16_deconv, self).__init__()
        self.channel = channel
        self.height = height
        self.width = width

        pool_list = [4, 9, 16, 23, 30]

        for index in pool_list:
            vgg16_pretrained.features[index].return_indices = True
        
        self.encoder1 = vgg16_pretrained.features[:4]
        self.pool1 = vgg16_pretrained.features[4]

        self.encoder2 = vgg16_pretrained.features[5:9]
        self.pool2 = vgg16_pretrained.features[9]

        self.encoder3 = vgg16_pretrained.features[10:16]
        self.pool3 = vgg16_pretrained.features[16]

        self.encoder4 = vgg16_pretrained.features[17:23]
        self.pool4 = vgg16_pretrained.features[23]

        self.encoder5 = vgg16_pretrained.features[24: 30]
        self.pool5 = vgg16_pretrained.features[30]

        self.classifier = nn.Sequential(
            torch.nn.Linear(num_linear, 4096),
            torch.nn.ReLU(),
            torch.nn.Linear(4096, num_linear),
            torch.nn.ReLU()
        ) 

        self.decoder5 = decoder(512, 512)
        self.unpool5 = nn.MaxUnpool2d(2, 2)
        
        self.decoder4 = decoder(512, 256)
        self.unpool4 = nn.MaxUnpool2d(2, 2)
        
        self.decoder3 = decoder(256, 128)
        self.unpool3 = nn.MaxUnpool2d(2, 2)

        self.decoder2 = decoder(128, 64, 2)
        self.unpool2 = nn.MaxUnpool2d(2, 2)

        self.decoder1 = decoder(64, num_classes, 2)
        self.unpool1 = nn.MaxUnpool2d(2, 2)
    
    def forward(self, x):
        print('x size:', x.size())

        encoder1 = self.encoder1(x)
        print('encoder1 size: ', encoder1.size())
        output_size1 = encoder1.size()        
        pool1, indices1 = self.pool1(encoder1)
        print('pool1 size: ', pool1.size(), 'indices1 size : ', indices1.size())

        encoder2 = self.encoder2(pool1)
        print('encoder2 size :', encoder2.size())
        output_size2 = encoder2.size()
        pool2, indices2 = self.pool2(encoder2)
        print('pool2 size : ', pool2.size() ,'indices2 size : ', indices2.size())

        encoder3 = self.encoder3(pool2)
        print('encoder3 size : ', encoder3.size())
        output_size3 = encoder3.size()
        pool3, indices3 = self.pool3(encoder3)
        print('pool3 size : ', pool3.size(), 'indices3 size : ', indices3.size())

        encoder4 = self.encoder4(pool3)
        print('encoder4 size : ', encoder4.size())
        output_size4 = encoder4.size()
        pool4, indices4 = self.pool4(encoder4)
        print('pool4 size : ', pool4.size(), 'indices4 size : ', indices4.size())

        encoder5 = self.encoder5(pool4)
        print('encoder5 size : ', encoder5.size())
        output_size5 = encoder5.size()
        pool5, indices5 = self.pool5(encoder5)
        print('pool5 size : ', pool5.size(), 'indices5 size : ', indices5.size())


        pool5 = pool5.view(pool5.size(0), -1)
        print('pool5 : ', pool5.size())
        fc = self.classifier(pool5)
        print('fc :', fc.size())
        fc = fc.reshape(1, self.channel, self.height, self.width)
        print('fc : ', fc.size())

        unpool5 = self.unpool5(input = fc, indices = indices5, output_size = output_size5)
        print('unpool5 : ', unpool5.size())
        decoder5 = self.decoder5(unpool5)
        print('decoder 5 : ', decoder5.size())

        unpool4 = self.unpool4(input = decoder5, indices = indices4, output_size = output_size4)
        print('unpool4 : ', unpool4.size())
        decoder4 = self.decoder4(unpool4)
        print('decoder 4 : ', decoder4.size())

        unpool3 = self.unpool3(input = decoder4, indices = indices3, output_size = output_size3)
        print('unpool3 : ', unpool3.size())
        decoder3 = self.decoder3(unpool3)
        print('decoder 3 : ', decoder3.size())

        unpool2 = self.unpool2(input = decoder3, indices = indices2, output_size = output_size2)
        print('unpool2 : ', unpool2.size())
        decoder2 = self.decoder2(unpool2)
        print('deocder 2 : ', decoder2.size())

        unpool1 = self.unpool1(input = decoder2, indices = indices1, output_size = output_size1)
        print('unpool1 : ', unpool1.size())
        decoder1 = self.decoder1(unpool1)
        print('decoder 1 : ', decoder1.size())

        return decoder1 


In [3]:
temp = torch.zeros(1, 3, 512, 512)
DeConvNet = VGG16_deconv(num_classes = 8, num_linear = 131072, channel = 512, height = 16, width = 16)
zz = DeConvNet(temp)
zz.size()

x size: torch.Size([1, 3, 512, 512])
encoder1 size:  torch.Size([1, 64, 512, 512])
pool1 size:  torch.Size([1, 64, 256, 256]) indices1 size :  torch.Size([1, 64, 256, 256])
encoder2 size : torch.Size([1, 128, 256, 256])
pool2 size :  torch.Size([1, 128, 128, 128]) indices2 size :  torch.Size([1, 128, 128, 128])
encoder3 size :  torch.Size([1, 256, 128, 128])
pool3 size :  torch.Size([1, 256, 64, 64]) indices3 size :  torch.Size([1, 256, 64, 64])
encoder4 size :  torch.Size([1, 512, 64, 64])
pool4 size :  torch.Size([1, 512, 32, 32]) indices4 size :  torch.Size([1, 512, 32, 32])
encoder5 size :  torch.Size([1, 512, 32, 32])
pool5 size :  torch.Size([1, 512, 16, 16]) indices5 size :  torch.Size([1, 512, 16, 16])
pool5 :  torch.Size([1, 131072])
fc : torch.Size([1, 131072])
fc :  torch.Size([1, 512, 16, 16])
unpool5 :  torch.Size([1, 512, 32, 32])
decoder 5 :  torch.Size([1, 512, 32, 32])
unpool4 :  torch.Size([1, 512, 64, 64])
decoder 4 :  torch.Size([1, 256, 64, 64])
unpool3 :  torch.Si

torch.Size([1, 8, 512, 512])